# 0. 구글 드라이브와 연결 및 각종 라이브러리 설정

### 구글 드라이브와 연결

In [7]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


### 라이브러리 설치

In [8]:
!pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [9]:
!pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [10]:
!pip install soynlp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### 라이브러리 불러오기

In [11]:
from konlpy.tag import Okt
from tqdm import tqdm
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import nltk
import re
from soynlp.normalizer import *
from konlpy.tag import Okt

# 1. 데이터 로드

### 데이터 로드

In [12]:
nsmc0 = pd.read_csv('/content/gdrive/MyDrive/CUAI 하계 컨퍼런스 프로젝트/data set/nsmc_data.txt', sep='\t')

### 가져온 데이터 확인

In [13]:
nsmc0.head(2)

,id,document,label
0,8112052,어릴때보고 지금다시봐도 재밌어요ㅋㅋ,1
1,8132799,"디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산...",1


### sample의 개수 확인

In [14]:
print('nsmc0 sample 수: {}개'.format(nsmc0.shape[0]))

nsmc0 sample 수: 200000개


# 2. 데이터 정제
### 중복된 값, 결측 값

### nsmc 데이터 확인 및 정제

`결측값 제거`

In [15]:
# 결측값을 확인

nsmc0.isnull().sum()

id          0
document    8
label       0
dtype: int64

In [16]:
nsmc0.loc[nsmc0['document'].isnull()]

,id,document,label
46471,6369843,NaN,1
60735,511097,NaN,1
77665,2172111,NaN,1
84098,402110,NaN,1
127017,5942978,NaN,0
172375,5026896,NaN,0
173526,1034280,NaN,0
197279,1034283,NaN,0


In [17]:
nsmc1 = nsmc0.dropna(how='any')

In [18]:
# 결측값을 확인

nsmc1.isnull().sum()

id          0
document    0
label       0
dtype: int64

`중복값 제거`

In [19]:
print('nsmc1 데이터 샘플의 수: {}'.format(nsmc1.shape[0]))
print('nsmc1 데이터의 document 종류의 수: {}'.format(nsmc1['document'].nunique()))

nsmc1 데이터 샘플의 수: 199992
nsmc1 데이터의 document 종류의 수: 194543


In [20]:
nsmc2 = nsmc1.drop_duplicates(subset=['document'], ignore_index=True)

In [21]:
print('nsmc2 데이터 샘플의 수: {}'.format(nsmc2.shape[0]))
print('nsmc2 데이터의 document 종류의 수: {}'.format(nsmc2['document'].nunique()))

nsmc2 데이터 샘플의 수: 194543
nsmc2 데이터의 document 종류의 수: 194543


`노이즈 제거`

In [22]:
nsmc2.loc[nsmc2['document'].map(lambda sent: 'ㄷ' in sent)]

,id,document,label
45,10142929,내가 남자라그런가 이거 겁나 긴장감있고 흥미진진하던데..ㄷㄷ 나만그런가워낙에 격투씬...,1
183,8484420,와 쩌네..ㄷㄷㄷ 캬..ㅎㅎ,1
224,8851783,내가무서운걸잘못보는데진짜무서워하면서도재미있어서잘봤음.박한별송지효조안다이쁘더라..조안...,1
353,8828360,과하지않다 강간범새끼들은 저것보다 더해줘도 분하다 강간은 정신을 완전히 난도질하는거...,1
513,10125993,"할머니 돌아가시고 우는데, 우는소리가 낯익다...네이티리ㅡ그녀였구나!!!어쩐지. 연...",1
...,...,...,...
193212,9045400,이감독의 다른영화도과연 스토리가 산인지 보고싶을정도로 소름끼친ㄷㅏ;; 2008년에 ...,0
193227,7497120,와 ㅋㅋ 대박 ㅋㅋㅋㅋ정말 앵간한 영화 다보는편인데 이거 정말주성치만 믿엇다 .. ...,0
193282,7865185,1점도 아깝ㄷ,0
194397,5501124,액션도 너무 가짜로 치는거 티가나고 ..무엇보다 배우들의 발 연기가 ㄷㄷ,0


In [64]:
nsmc2.loc[nsmc2['document'].map(lambda sent: 'ㅋ' in sent)]['label'].value_counts()

1    7199
0    5406
Name: label, dtype: int64

In [65]:
p = re.compile('[ㅋ]{2,}')

In [66]:
kind = []
for i in range(nsmc2.shape[0]):
   kind.extend(p.findall(nsmc2['document'][i]))

for c in list(set(kind)):
  print('{}의 수: {}'.format(c, kind.count(c)))

ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ의 수: 6
ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ의 수: 2
ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ의 수: 6
ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ의 수: 1
ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ의 수: 1
ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ의 수: 70
ㅋㅋㅋㅋㅋㅋㅋ의 수: 156
ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ의 수: 2
ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ의 수: 1
ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ의 수: 8
ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ의 수: 2
ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ의 수: 3
ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ의 수: 33
ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ의 수: 1
ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ의 수: 1
ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ의 수: 1
ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ의 수: 1
ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ

In [26]:
nsmc2

,id,document,label
0,8112052,어릴때보고 지금다시봐도 재밌어요ㅋㅋ,1
1,8132799,"디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산...",1
2,4655635,폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.,1
3,9251303,와.. 연기가 진짜 개쩔구나.. 지루할거라고 생각했는데 몰입해서 봤다.. 그래 이런...,1
4,10067386,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화.,1
...,...,...,...
194538,8963373,포켓 몬스터 짜가 ㅡㅡ;;,0
194539,3302770,쓰.레.기,0
194540,5458175,완전 사이코영화. 마지막은 더욱더 이 영화의질을 떨어트린다.,0
194541,6908648,왜난 재미없었지 ㅠㅠ 라따뚜이 보고나서 스머프 봐서 그런가 ㅋㅋ,0


In [27]:
nsmc3 = nsmc2.copy()
nsmc3

,id,document,label
0,8112052,어릴때보고 지금다시봐도 재밌어요ㅋㅋ,1
1,8132799,"디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산...",1
2,4655635,폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.,1
3,9251303,와.. 연기가 진짜 개쩔구나.. 지루할거라고 생각했는데 몰입해서 봤다.. 그래 이런...,1
4,10067386,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화.,1
...,...,...,...
194538,8963373,포켓 몬스터 짜가 ㅡㅡ;;,0
194539,3302770,쓰.레.기,0
194540,5458175,완전 사이코영화. 마지막은 더욱더 이 영화의질을 떨어트린다.,0
194541,6908648,왜난 재미없었지 ㅠㅠ 라따뚜이 보고나서 스머프 봐서 그런가 ㅋㅋ,0


In [28]:
nsmc3['document'] = nsmc3['document'].map(lambda sent: re.sub('ㅜ', 'ㅠ', sent))

In [29]:
nsmc3    # 'ㅜ'를 'ㅠ'로 바꾼 data

,id,document,label
0,8112052,어릴때보고 지금다시봐도 재밌어요ㅋㅋ,1
1,8132799,"디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산...",1
2,4655635,폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.,1
3,9251303,와.. 연기가 진짜 개쩔구나.. 지루할거라고 생각했는데 몰입해서 봤다.. 그래 이런...,1
4,10067386,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화.,1
...,...,...,...
194538,8963373,포켓 몬스터 짜가 ㅡㅡ;;,0
194539,3302770,쓰.레.기,0
194540,5458175,완전 사이코영화. 마지막은 더욱더 이 영화의질을 떨어트린다.,0
194541,6908648,왜난 재미없었지 ㅠㅠ 라따뚜이 보고나서 스머프 봐서 그런가 ㅋㅋ,0


In [30]:
nsmc4 = nsmc3.copy()

In [31]:
nsmc4['document'] = nsmc4['document'].map(lambda sent: re.sub('[^ㅋㅎㅡㅉㅠ 가-힣]', '', sent))

In [32]:
nsmc4    # 영어와 특수문자는 지워준 data

,id,document,label
0,8112052,어릴때보고 지금다시봐도 재밌어요ㅋㅋ,1
1,8132799,디자인을 배우는 학생으로 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산업...,1
2,4655635,폴리스스토리 시리즈는 부터 뉴까지 버릴께 하나도 없음 최고,1
3,9251303,와 연기가 진짜 개쩔구나 지루할거라고 생각했는데 몰입해서 봤다 그래 이런게 진짜 영화지,1
4,10067386,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화,1
...,...,...,...
194538,8963373,포켓 몬스터 짜가 ㅡㅡ,0
194539,3302770,쓰레기,0
194540,5458175,완전 사이코영화 마지막은 더욱더 이 영화의질을 떨어트린다,0
194541,6908648,왜난 재미없었지 ㅠㅠ 라따뚜이 보고나서 스머프 봐서 그런가 ㅋㅋ,0


In [33]:
nsmc5 = nsmc4.copy()

In [34]:
nsmc5

,id,document,label
0,8112052,어릴때보고 지금다시봐도 재밌어요ㅋㅋ,1
1,8132799,디자인을 배우는 학생으로 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산업...,1
2,4655635,폴리스스토리 시리즈는 부터 뉴까지 버릴께 하나도 없음 최고,1
3,9251303,와 연기가 진짜 개쩔구나 지루할거라고 생각했는데 몰입해서 봤다 그래 이런게 진짜 영화지,1
4,10067386,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화,1
...,...,...,...
194538,8963373,포켓 몬스터 짜가 ㅡㅡ,0
194539,3302770,쓰레기,0
194540,5458175,완전 사이코영화 마지막은 더욱더 이 영화의질을 떨어트린다,0
194541,6908648,왜난 재미없었지 ㅠㅠ 라따뚜이 보고나서 스머프 봐서 그런가 ㅋㅋ,0


In [35]:
nsmc5['document'] = nsmc5['document'].map(lambda sent: emoticon_normalize(sent, num_repeats=2))

In [36]:
nsmc5['document'] = nsmc5['document'].map(lambda sent: re.sub('ㅋㅋㅋ','ㅋㅋ',sent))
nsmc5['document'] = nsmc5['document'].map(lambda sent: re.sub('ㅎㅎㅎ','ㅎㅎ',sent))
nsmc5['document'] = nsmc5['document'].map(lambda sent: re.sub('ㅠㅠㅠ','ㅠㅠ',sent))
nsmc5['document'] = nsmc5['document'].map(lambda sent: re.sub('ㅡㅡㅡ','ㅡㅡ',sent))
nsmc5['document'] = nsmc5['document'].map(lambda sent: re.sub('ㅉㅉㅉ','ㅉㅉ',sent))

In [37]:
nsmc6 = nsmc5.copy()


In [38]:
nsmc6['document'] = nsmc6['document'].map(lambda sent: re.sub('^ +', '', sent))

In [39]:
nsmc6['document'].replace('', np.nan, inplace=True)

In [40]:
nsmc6

,id,document,label
0,8112052,어릴때보고 지금다시봐도 재밌어ㅋㅋ,1
1,8132799,디자인을 배우는 학생으로 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산업...,1
2,4655635,폴리스스토리 시리즈는 부터 뉴까지 버릴께 하나도 없음 최고,1
3,9251303,와 연기가 진짜 개쩔구나 지루할거라고 생각했는데 몰입해서 봤다 그래 이런게 진짜 영화지,1
4,10067386,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화,1
...,...,...,...
194538,8963373,포켓 몬스터 짜가 ㅡㅡ,0
194539,3302770,쓰레기,0
194540,5458175,완전 사이코영화 마지막은 더욱더 이 영화의질을 떨어트린다,0
194541,6908648,왜난 재미없었지 ㅠㅠ 라따뚜이 보고나서 스머프 봐서 그런가 ㅋㅋ,0


In [41]:
nsmc3.loc[nsmc6['document'].isnull()]

,id,document,label
202,10216462,goodgoodgoodgoodgoodgoodgood,1
607,8085167,Very interesting,1
660,3579401,Killing time yong good,1
982,5338464,GOOD~,1
1197,7887700,♥♥,1
...,...,...,...
193851,4852134,.............................,0
193876,9233162,Real Trash Movie,0
193899,4602734,2,0
194144,2259844,ㄲ ㅓ ㅈ ㅕ,0


In [42]:
print(nsmc5['document'].isnull().sum())
print(nsmc6['document'].isnull().sum())

# 주의!!!! null 값이 Nan으로 되어있지 않을 수 있다!!

0
1108


In [43]:
nsmc7 = nsmc6.copy()

In [44]:
nsmc7 = nsmc7.dropna(how='any')

In [45]:
print(nsmc7['document'].isnull().sum())

# 결측값이 모두 사라졌다.

0


In [46]:
nsmc7

,id,document,label
0,8112052,어릴때보고 지금다시봐도 재밌어ㅋㅋ,1
1,8132799,디자인을 배우는 학생으로 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산업...,1
2,4655635,폴리스스토리 시리즈는 부터 뉴까지 버릴께 하나도 없음 최고,1
3,9251303,와 연기가 진짜 개쩔구나 지루할거라고 생각했는데 몰입해서 봤다 그래 이런게 진짜 영화지,1
4,10067386,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화,1
...,...,...,...
194538,8963373,포켓 몬스터 짜가 ㅡㅡ,0
194539,3302770,쓰레기,0
194540,5458175,완전 사이코영화 마지막은 더욱더 이 영화의질을 떨어트린다,0
194541,6908648,왜난 재미없었지 ㅠㅠ 라따뚜이 보고나서 스머프 봐서 그런가 ㅋㅋ,0


In [47]:
nsmc8 = nsmc7.copy()
nsmc8['document'].nunique()

# 겹치는 것이 존재한다.

190368

In [48]:
nsmc8 = nsmc8.drop_duplicates(subset=['document', 'label'], ignore_index=True)

In [49]:
nsmc8['document'].shape[0], nsmc8['document'].nunique()

# document는 같지만 label이 다른 것이 존재하는 듯..(버리지 않는 것이 좋으려나?)

(190530, 190368)

In [50]:
nsmc8

,id,document,label
0,8112052,어릴때보고 지금다시봐도 재밌어ㅋㅋ,1
1,8132799,디자인을 배우는 학생으로 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산업...,1
2,4655635,폴리스스토리 시리즈는 부터 뉴까지 버릴께 하나도 없음 최고,1
3,9251303,와 연기가 진짜 개쩔구나 지루할거라고 생각했는데 몰입해서 봤다 그래 이런게 진짜 영화지,1
4,10067386,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화,1
...,...,...,...
190525,4834376,개연성이 없어요 별루다,0
190526,8963373,포켓 몬스터 짜가 ㅡㅡ,0
190527,5458175,완전 사이코영화 마지막은 더욱더 이 영화의질을 떨어트린다,0
190528,6908648,왜난 재미없었지 ㅠㅠ 라따뚜이 보고나서 스머프 봐서 그런가 ㅋㅋ,0


In [51]:
# 띄어쓰기+맞춤법 교정, 영화->강의
# 한 번 생각은 해볼만 하다.

In [52]:
!pip install git+https://github.com/ssut/py-hanspell.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/ssut/py-hanspell.git to /tmp/pip-req-build-zndqu8x2
  Running command git clone -q https://github.com/ssut/py-hanspell.git /tmp/pip-req-build-zndqu8x2
  Created wheel for py-hanspell: filename=py_hanspell-1.1-py3-none-any.whl size=4868 sha256=aea3479dacd8882f5d6210470cdcb63d5fb89805c1bf44d534ab67ff7efd59b4
  Stored in directory: /tmp/pip-ephem-wheel-cache-h1rwmq04/wheels/ab/f5/7b/d4124bb329c905301baed80e2ae45aa14e824f62ebc3ec2cc4
Successfully built py-hanspell


In [53]:
from hanspell import spell_checker
spell_checker.check(u'완전 사이코영화 마지망은더욱더 이 영화의질을 떨어뜨린다').checked

'완전 사이코 영화 마지막은 더욱더 이 영화의 질을 떨어뜨린다'

In [79]:
def spell_check(sent):
  return spell_checker.check(sent).checked

spell_check('')

'그나마 다행인 게로 본 거 그런데도 전기세가 아꿉감독은 먼 생각으로 영화를 만들었을까 액션 영화도 안 봤을까 진심으로 한국 영화의 퇴보를 부르는 영화 극장 가서 본 사람들 얼마나 억울했을까'

In [56]:
nsmc9 = nsmc8.copy()   # nsmc9에 각자 맞춤법 검사 한 것을 합칠 것이다.

In [57]:
# nsmc9['document'] = nsmc9['document'].map(spell_check)

In [58]:
# nsmc9['document'].iloc[:5] = nsmc9['document'].iloc[:5].map(spell_check)

In [59]:
# CJ = nsmc9.loc[:63000].copy()
# MK = nsmc9.loc[63001:126000].copy()
# KM = nsmc9.loc[126001:].copy()

In [60]:
# CJ.to_excel('/content/gdrive/MyDrive/CUAI 하계 컨퍼런스 프로젝트/data preprocessing 최종/CJ.xlsx')

In [61]:
# MK.to_excel('/content/gdrive/MyDrive/CUAI 하계 컨퍼런스 프로젝트/data preprocessing 최종/MK.xlsx')

In [62]:
# KM.to_excel('/content/gdrive/MyDrive/CUAI 하계 컨퍼런스 프로젝트/data preprocessing 최종/KM.xlsx')

In [80]:
nsmc9_1 = pd.read_excel('/content/gdrive/MyDrive/CUAI 하계 컨퍼런스 프로젝트/data preprocessing 최종/done_CJ.xlsx', engine='openpyxl')
nsmc9_2 = pd.read_excel('/content/gdrive/MyDrive/CUAI 하계 컨퍼런스 프로젝트/data preprocessing 최종/done_KM.xlsx', engine='openpyxl')
nsmc9_3 = pd.read_excel('/content/gdrive/MyDrive/CUAI 하계 컨퍼런스 프로젝트/data preprocessing 최종/done_MK.xlsx', engine='openpyxl')

In [81]:
nsmc9_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63001 entries, 0 to 63000
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  63001 non-null  int64 
 1   document    63001 non-null  object
 2   label       63001 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.4+ MB


In [82]:
nsmc9_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64529 entries, 0 to 64528
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  64529 non-null  int64 
 1   id          64529 non-null  int64 
 2   document    64529 non-null  object
 3   label       64529 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 2.0+ MB


In [83]:
nsmc9_3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63000 entries, 0 to 62999
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Unnamed: 0    63000 non-null  int64 
 1   Unnamed: 0.1  63000 non-null  int64 
 2   id            63000 non-null  int64 
 3   document      63000 non-null  object
 4   label         63000 non-null  int64 
dtypes: int64(4), object(1)
memory usage: 2.4+ MB


In [84]:
nsmc9 = pd.concat([nsmc9_1, nsmc9_2, nsmc9_3], ignore_index=True)

In [85]:
nsmc9.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 190530 entries, 0 to 190529
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Unnamed: 0    190530 non-null  int64  
 1   document      190530 non-null  object 
 2   label         190530 non-null  int64  
 3   id            127529 non-null  float64
 4   Unnamed: 0.1  63000 non-null   float64
dtypes: float64(2), int64(2), object(1)
memory usage: 7.3+ MB


In [86]:
nsmc9 = nsmc9[['document', 'label']]
nsmc9

,document,label
0,어릴 때 보고 지금 다시 봐도 재밌어ㅋㅋ,1
1,디자인을 배우는 학생으로 외국 디자이너와 그들이 일군 전통을 통해 발전해가는 문화산...,1
2,폴리스 스토리 시리즈는부터 뉴까지 버릴게 하나도 없음 최고,1
3,와 연기가 진짜 개쩔구나 지루할 거라고 생각했는데 몰입해서 봤다 그래 이런 게 진짜...,1
4,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화,1
...,...,...
190525,소재가 후지다는 말이 아니다 그냥 감독의 역량이 부족하다 마치 웃겨보라 했더니 여장...,0
190526,박중훈 그거 참 좋아하게 생겼다 마누라도 일본 여자 박중훈 캐스팅 자제 바람 식상하...,0
190527,이런 영화에 점주는 사람들은 도대체 뭡니까,0
190528,뭔가 감동적인 걸 만들려 했겠지만 연출 시나리오 연기 다 별로,0


In [87]:
nsmc9.to_excel('/content/gdrive/MyDrive/CUAI 하계 컨퍼런스 프로젝트/data preprocessing 최종/nsmc_preprocessed.xlsx')    # 최종 nsmc 데이터